# Transforming Temporal-Dynamic Graphs Into Time-Series Data for Solving Event Detection Problems

Event detection problems on temporal-dynamic graphs aim to detect important events by detection abnormal changes on the
network. Because of the excessive use of social media, many real world problems can be modelled as temporal-dynamic graph
data. With the recent progress in graph representation learning, new anomaly detection on static graphs are studied. In this
work, we present a workflow for event detection on temporal-dynamic graphs with using graph representation learning.
Our workflow uses generated embeddings of the temporal-dynamic graph to transform the problem into a unsupervised
time-series anomaly detection problem. Since this is a widely studied research area, transforming temporal-dynamic graph
data into multivariate time series data, provides many possible solutions for the event detection problems. We have evaluated
our proposed workflow on four different real-world datasets and compared our results. Our workflow shows competitive per-
formance, when compared to previous studies. This study gives a proof of concept for using graph embeddings as time-series
data in anomaly detection task.

# Proposed Workflow

In the figure bellow you can see the proposed model workflow. Input is a temporal-dynamic graph G which consists of static
snapshots of the graph taken in different time steps.Then model generates n-dimensional vector embeddings from given
input graph, with using graph representation learning. After this step model pass these embeddings to an unsupervised
anomaly detector. Output of proposed workflow is the anomaly scores corresponding to each time step.

<img src="Proposed_Workflow.png">

In following experiments, we are going to use our proposed workflow. After pre-processing our data, first step is
to generate graph embeddings. For this task we used tdGraphEmbed model. Model generates 40 random-walks
for each node in the graph and length of each walk is 16 in our experiments. The model is trained 50 iterations,
with generated random-walk document. In the second step we are going to use time-series anomaly detectors we
mentioned above. For these algorithms we used Merlion machine learning library for time series data.

# Importing Packages

Before importing libraries, make sure to install requirements in Github repository.

In [1]:
from gensim.models.doc2vec import Doc2Vec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from merlion.utils import TimeSeries
from evaluation_util import *

from tdGraphEmbed.tdGraphEmbed.temporal_graph import TemporalGraph
from tdGraphEmbed.tdGraphEmbed.model import TdGraphEmbed
from datasetConverter import dataset_convert
from tempfile import TemporaryFile

import datetime

C:\Users\TR\anaconda3\envs\tdGraphEmbed\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generating Temporal-Dynamic Graph Embeddings

In this step we are going to read the data from files and generate a model for training process. For getting the datasets you, we will use the data_conver() function. Available datasets are:

Tw-WorldCup - The Twitter WorldCup datasets. In experiments you can use granularity as hours.

Tw-Terror-Security - The Twitter Terror Security. In experiments you can use granularity as days.

gameofthrones - The Reddit Game of Thrones. You can read directly, since this dataset is provided as picke file.

formula - The Reddit Formula 1. You can read directly, since this dataset is provided as picke file.

In [2]:
graphs = dataset_convert(dataset="Tw-WorldCup",granularity="hours")
model = TdGraphEmbed(dataset_name="Tw-WorldCup")

In [4]:
documents = model.get_documents_from_graph(graphs)

Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 207.26it/s]


Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 952.44it/s]


Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 634.93it/s]


Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 470.60it/s]


Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 283.69it/s]


...

Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 300.75it/s]


Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 310.08it/s]


Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 294.12it/s]


Computing transition probabilities: 100%|██████████| 51/51 [00:00<00:00, 6375.84it/s]


Generating walks (CPU: 1): 100%|██████████| 40/40 [00:00<00:00, 570.68it/s]


In [5]:
model.run_doc2vec(documents)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
Model Saved


In [6]:
graph_vectors = model.get_embeddings()
np.save("tdGraphEmbed/saved_embeddings/Tw-WorldCup.npy", graph_vectors)

In the code above you can read the dataset and train the graph representation learning model. Then save the model and embeddings into files. Training for this process takes a long time around 15-24 hours to complete. Because of this we will use the model and files we have saved.

# Unsupervised Time Series Anomaly Detection

Bellow, we will read the saved model and dataset labels and prepares the data in the format of Merlion library. Saved model file will provide us with time-stamps and n-dimensional embeddings and we will use the labels to evaluate our model. All training process is fully unsupervised and we only use labels for evaluation.

In [2]:
model_path = "tdGraphEmbed/trained_models/Tw-WorldCup.model"
labels_path = "Datasets/Twitter_WorldCup/Twitter_WorldCup_2014_labels.txt"

#model_path = "tdGraphEmbed/trained_models/Tw-Terror-Security.model"
#labels_path = "Datasets/Twitter_Security/Twitter_May_Aug_2014_TerrorSecurity_labels.txt"

#model_path = "tdGraphEmbed/trained_models/GoT-2017.model"
#labels_path = "Datasets/gameofthrones/gameofthrones_2017_labels.txt"

#model_path = "tdGraphEmbed/trained_models/Formula-2019.model"
#labels_path = "Datasets/formula/formula_2019_labels.txt"



model = Doc2Vec.load(model_path)
doc_vecs = model.docvecs.doctag_syn0
doc_vecs = doc_vecs[np.argsort([model.docvecs.index_to_doctag(i) for i in range(0, doc_vecs.shape[0])])]

time_stamps = list(model.docvecs.doctags.keys())
time_series_custom = pd.DataFrame(doc_vecs, index=time_stamps)

ls = readFiles(labels_path, granularity="hours")
df_metadata = pd.DataFrame(columns = ['trainval', 'anomaly'], index = time_stamps)
df_metadata = generate_metadata(df_metadata, time_stamps, ls)

Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).


In this step we are going to train our unsupervised time-series anomaly detection model. There are some different settings for different datasets. Bellow the settings are adjusted for The Twitter World-Cup dataset. Overall important parameters are; "top" variable is the k variable in Recall@k and Precision@k and defines the amount of anomalies to detect. Available models we recommend are "VAE", "LSTMED", and "IsolationForest". For other datasets please follow the comments.

In [6]:
from merlion.models.factory import ModelFactory
from merlion.post_process.threshold import AggregateAlarms

# @k parameter to set
top=5

train_data = TimeSeries.from_pd(time_series_custom[:])
test_labels = TimeSeries.from_pd(df_metadata["anomaly"][:])

#Available models are VAE, LSTMED, and IsolationForest
#Because of the data properties Isolation Forest is best in Twitter Security dataset.
#It is best to use VAE or LSTMED on other datasets.
model = ModelFactory.create("VAE",
                            threshold=AggregateAlarms(alm_threshold=0))

model.train(train_data)
labels = model.get_anomaly_label(train_data)
df_temp = labels.to_pd()
df_cpy = df_temp.copy()

#For datasets other than The Twitter World-Cup ascending should be True. This is due to some
#implementation error on Merlion library.
df_temp = get_top_anomalies(df_temp,ascending=False , top=top)

#If you are using Isolation Forest Model you have to use test_labels_temp = test_labels[1:]
#Because ısolation forest does nor return an anomaly score for first time-stamp.
test_labels_temp = test_labels[:]

 |████████████████████████████████████████| 100.0% Complete, Loss 0.0013
Anomaly Threshold: 
anom_score    2.435622
Name: 2014-07-13 18:00:00, dtype: float64


# Evaluation

In this part you can evaluate the model with precision and recall. Also you can try out different delay factors. Since model training is trivial. You can get different results in significance interval.

In [7]:
prec = get_precision(df_temp, test_labels_temp,delay=0)
rec = get_recall(df_temp, test_labels_temp,delay=0,top=top)
acc = get_accuracy(df_temp, test_labels_temp)
print("Top",top,"time-stamps are considered as anomalies")
print("Precision:", prec)
print("Recall:", rec)
print("Accuracy:", acc)

Top 5 time-stamps are considered as anomalies
Precision: 0.6
Recall: 0.025210084033613446
Accuracy: 0.9341150195421553


In [8]:
for i in range(6):  
    prec = get_precision(df_temp, test_labels_temp,delay=i)
    rec = get_recall(df_temp, test_labels_temp,delay=i,top=top)
    print("Precision and Recall with delay",i)
    print("Precision:", prec)
    print("Recall:", rec)
    print("---------")
    

Precision and Recall with delay 0
Precision: 0.6
Recall: 0.025210084033613446
---------
Precision and Recall with delay 1
Precision: 1.0
Recall: 0.07563025210084033
---------
Precision and Recall with delay 2
Precision: 1.0
Recall: 0.1092436974789916
---------
Precision and Recall with delay 3
Precision: 1.0
Recall: 0.12605042016806722
---------
Precision and Recall with delay 4
Precision: 1.0
Recall: 0.14285714285714285
---------
Precision and Recall with delay 5
Precision: 1.0
Recall: 0.15966386554621848
---------
